In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

In [19]:
def parse_ria_news(date_from=None, max_news=None):
    """
    Парсит новости с сайта РИА Новости.
    
    Args:
        date_from (str, optional): Дата в формате 'YYYY-MM-DD'. Парсинг новостей начиная с этой даты.
        max_news (int, optional): Максимальное количество новостей для парсинга.
    
    Returns:
        pd.DataFrame: DataFrame с колонками ['date', 'title', 'text', 'link'].
    """
    
    base_url = "https://ria.ru"
    section_url = "https://ria.ru/world/"  # можно выбрать любой раздел
    news_list = []

    response = requests.get(section_url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Находим все карточки новостей
    news_cards = soup.find_all("div", class_="list-item__content")
    
    for card in news_cards:
        # Заголовок
        title_tag = card.find("a", class_="list-item__title")
        if not title_tag:
            continue
        title = title_tag.text.strip()
        link = title_tag['href']

        # Дата новости
        date_tag = card.find("span", class_="list-item__date")
        if date_tag:
            date_text = date_tag.text.strip()
            try:
                # Преобразуем дату в datetime
                news_date = datetime.strptime(date_text, "%d.%m.%Y %H:%M")
            except:
                news_date = None
        else:
            news_date = None

        # Проверка даты
        if date_from:
            date_from_dt = datetime.strptime(date_from, "%Y-%m-%d")
            if news_date and news_date < date_from_dt:
                continue

        # Получаем текст новости с внутренней страницы
        news_response = requests.get(link)
        news_soup = BeautifulSoup(news_response.text, "html.parser")

        # Дата новости на странице самой новости
        date_match = re.search(r'/(\d{8})/', link)
        if date_match:
            news_date = datetime.strptime(date_match.group(1), "%Y%m%d")
        else:
            news_date = None

        # Текст новости
        text_tag = news_soup.find("div", class_="article__body")
        text = text_tag.get_text(separator="\n").strip() if text_tag else ""

        news_list.append({
            "date": news_date,
            "title": title,
            "text": text,
            "link": link
        })

        # Проверка лимита
        if max_news and len(news_list) >= max_news:
            break

    df = pd.DataFrame(news_list)
    return df


In [20]:
df_news = parse_ria_news(date_from="2025-10-01", max_news=5)
print(df_news)

        date                                              title  \
0 2025-10-10  Посол России прокомментировал попытки Киева ди...   
1 2025-10-10  Страны ЕС изменят процедуру пересечения границ...   
2 2025-10-10  США опровергли заявления о поставках Пакистану...   
3 2025-10-10  В Тегеране проходит шествие в честь перемирия ...   
4 2025-10-10                        Путин встретился с Токаевым   

                                                text  \
0  МОСКВА, 10 окт - РИА Новости.\n Попытки киевск...   
1  БРЮССЕЛЬ, 10 окт – РИА Новости.\n Страны Еврос...   
2  НЬЮ-ДЕЛИ, 10 окт – РИА Новости.\n Посольство С...   
3  ТЕГЕРАН, 10 окт - РИА Новости.\n Сотни жителей...   
4  ДУШАНБЕ, 10 окт — РИА Новости. \nПрезидент Рос...   

                                                link  
0      https://ria.ru/20251010/posol-2047525088.html  
1  https://ria.ru/20251010/evrosoyuz-2047524904.html  
2     https://ria.ru/20251010/rakety-2047523321.html  
3  https://ria.ru/20251010/peremirie-204

In [21]:
df_news

,date,title,text,link
0,2025-10-10,Посол России прокомментировал попытки Киева ди...,"МОСКВА, 10 окт - РИА Новости.\n Попытки киевск...",https://ria.ru/20251010/posol-2047525088.html
1,2025-10-10,Страны ЕС изменят процедуру пересечения границ...,"БРЮССЕЛЬ, 10 окт – РИА Новости.\n Страны Еврос...",https://ria.ru/20251010/evrosoyuz-2047524904.html
2,2025-10-10,США опровергли заявления о поставках Пакистану...,"НЬЮ-ДЕЛИ, 10 окт – РИА Новости.\n Посольство С...",https://ria.ru/20251010/rakety-2047523321.html
3,2025-10-10,В Тегеране проходит шествие в честь перемирия ...,"ТЕГЕРАН, 10 окт - РИА Новости.\n Сотни жителей...",https://ria.ru/20251010/peremirie-2047522787.html
4,2025-10-10,Путин встретился с Токаевым,"ДУШАНБЕ, 10 окт — РИА Новости. \nПрезидент Рос...",https://ria.ru/20251010/vstrecha-2047517614.html


In [15]:
print(df_news.loc[0].title, df_news.loc[0].text, df_news.loc[0].link, sep="\n")

США опровергли заявления о поставках Пакистану ракет средней дальности
НЬЮ-ДЕЛИ, 10 окт – РИА Новости.
 Посольство США в Индии опровергло заявления о якобы планируемых поставках Пакистану новых усовершенствованных ракет класса "воздух-воздух" средней дальности (AMRAAM, Advanced Medium-Range Air-to-Air Missile). 
"Тридцатого сентября 2025 года министерство обороны США опубликовало список стандартных объявлений о контрактах, в которых упоминалось изменение существующего контракта на поставку вооружений и запасных частей для ряда стран, включая 
Пакистан
. Администрация хотела бы подчеркнуть, что, вопреки ложным сообщениям СМИ, упомянутое изменение контракта не касается поставок Пакистану новых усовершенствованных ракет класса "воздух-воздух" средней дальности (AMRAAM)", - сообщило посольство. 
На Западе назвали 15 стран, способных нанести ядерный удар
18 сентября, 17:22
Там отметили, что поставка вооружений не предусматривает модернизации каких-либо существующих возможностей Пакистана. 
